In [1]:
import visa
import sys
sys.path.append('../')

import devices.TDS2024C
import PyCapture2 as pc2
import devices.Camera
import daq
import file

In [5]:
rm = visa.ResourceManager('@py')
print(rm.list_resources())

('ASRL/dev/ttyS0::INSTR', 'ASRL/dev/ttyACM1::INSTR', 'USB0::1689::934::C046401::0::INSTR')


In [3]:
bus = pc2.BusManager()
cam = bus.discoverGigECameras()

In [4]:
print(cam[0].serialNumber)
print(cam[0].modelName)

17571186
b'Blackfly BFLY-PGE-50A2M'


In [10]:
import re
import subprocess
device_re = re.compile(b"Bus (?P<bus>\d+) Device (?P<device>\d+): ID (?P<id>\w+:\w+) (?P<tag>.*)", re.I)
df = subprocess.check_output("lsusb")

devices = []
for i in df.split(b'\n'):
    if i:
        print(str(i))
        info = device_re.match(i)
        if info:
            dinfo = info.groupdict()
            dinfo['device'] = '/dev/bus/usb/%s/%s' % (dinfo.pop('bus'), dinfo.pop('device'))
            devices.append(dinfo)
print(devices)
# USB id 0416:5011 is the KA3005P power supply
# USB id 0699:03a6 is the TDS2024C scope


b'Bus 002 Device 001: ID 1d6b:0003 Linux Foundation 3.0 root hub'
b'Bus 001 Device 003: ID 046d:c31c Logitech, Inc. Keyboard K120'
b'Bus 001 Device 002: ID 046d:c52b Logitech, Inc. Unifying Receiver'
b'Bus 001 Device 015: ID 0699:03a6 Tektronix, Inc. '
b'Bus 001 Device 017: ID 2457:1012  '
b'Bus 001 Device 014: ID 0416:5011 Winbond Electronics Corp. Virtual Com Port'
b'Bus 001 Device 008: ID 2109:2817 VIA Labs, Inc. '
b'Bus 001 Device 007: ID 2109:2817 VIA Labs, Inc. '
b'Bus 001 Device 006: ID 2109:2817 VIA Labs, Inc. '
b'Bus 001 Device 005: ID 1a40:0101 Terminus Technology Inc. Hub'
b'Bus 001 Device 004: ID 1a40:0101 Terminus Technology Inc. Hub'
b'Bus 001 Device 001: ID 1d6b:0002 Linux Foundation 2.0 root hub'
[{'id': b'1d6b:0003', 'tag': b'Linux Foundation 3.0 root hub', 'device': "/dev/bus/usb/b'002'/b'001'"}, {'id': b'046d:c31c', 'tag': b'Logitech, Inc. Keyboard K120', 'device': "/dev/bus/usb/b'001'/b'003'"}, {'id': b'046d:c52b', 'tag': b'Logitech, Inc. Unifying Receiver', 'device

In [4]:
import serial as ser
import serial.tools.list_ports as prtlst

global COMs
COMs=[]
def getCOMs():
    global COMs
    pts= prtlst.comports()

    for pt in pts:
        print(pt)
        if 'USB' in pt[1]: #check 'USB' string in device description
            COMs.append(pt[0])
getCOMs()
print(COMs)

/dev/ttyS0 - ttyS0
/dev/ttyACM0 - ttyACM0
[]


In [22]:
import seabreeze.spectrometers as sb
devices = sb.list_devices()
print(devices[0])
print(devices[0].serial)

<SeaBreezeDevice HR4000:michaelito>
michaelito


'michaelito'

In [48]:
import os
ret = os.system("ping -c 1 169.254.248.180")
print(ret)

0


In [52]:
instr = {}
# First scan for cameras
cams = pc2.BusManager().discoverGigECameras()
for cam in cams:
    instr[cam.serialNumber] = {
                    'type' : 'Camera',
                    'adr' : cam.serialNumber,
                    'model' : cam.modelName
                    }
# Scan for USB devices using pyVisa
rm = visa.ResourceManager('@py')
visaDevices = rm.list_resources()
for name in visaDevices:
    if name == 'USB0::1689::934::C046401::0::INSTR':
        instr['C046401'] = {
                    'type' : 'TDS2024C',
                    'adr' : name,
                    'model' : 'TDS2024C'
                        }
# Find all serial over USB ports
pts = prtlst.comports()
for pt in pts:
    if 'USB' in pt[1]: # check 'USB' string in device description
        # Ping for the device ID to see what it is
        dev = ser.Serial(pt[0],
                                    baudrate=9600,
                                    bytesize=8,
                                    parity='N',
                                    stopbits=1,
                                    timeout=1)
        dev.write(b"*IDN?")
        ID = dev.read(16).decode("utf-8")
        if ID == 'KORADKA3005PV2.0':
            adr = pt[0].split('/')[-1]
            instr[adr] = {
                    'type' : 'KA3005P',
                    'adr' : adr,
                    'model' : 'KA3005P'
                        }
# Find all the connected spectrometers
devices = sb.list_devices()
for dev in devices:
    instr[dev.serial] = {
                    'type' : dev.model,
                    'adr' : dev.serial,
                    'model' : dev.model
                        }
# Ping the signal delay generator
ret = os.system("ping -c 1 169.254.248.180")
if ret == 0:
    instr[5025] = {
                    'type' : 'SRSDG645',
                    'adr' : 5025,
                    'model' : 'SRSDG645'
                        }

print(instr)

{17583372: {'type': 'Camera', 'adr': 17583372, 'model': b'Blackfly BFLY-PGE-31S4M'}, 17570564: {'type': 'Camera', 'adr': 17570564, 'model': b'Blackfly BFLY-PGE-13S2M'}, 17529184: {'type': 'Camera', 'adr': 17529184, 'model': b'Blackfly BFLY-PGE-50S5M'}, 17571186: {'type': 'Camera', 'adr': 17571186, 'model': b'Blackfly BFLY-PGE-50A2M'}, 'C046401': {'type': 'TDS2024C', 'adr': 'USB0::1689::934::C046401::0::INSTR', 'model': 'TDS2024C'}, 'ttyACM1': {'type': 'KA3005P', 'adr': 'ttyACM1', 'model': 'KA3005P'}, 'ttyACM0': {'type': 'KA3005P', 'adr': 'ttyACM0', 'model': 'KA3005P'}, 'michaelito': {'type': 'HR4000', 'adr': 'michaelito', 'model': 'HR4000'}, 5025: {'type': 'SRSDG645', 'adr': 5025, 'model': 'SRSDG645'}}


In [1]:
import serial as ser
import serial.tools.list_ports as prtlst
pts = prtlst.comports()
for pt in pts:
    print(pt[0])

/dev/ttyS0
/dev/ttyUSB0
/dev/ttyACM1
/dev/ttyACM0


In [2]:
pts[3].serial_number

'5573631333835150F150'

In [32]:
print(pts[1].vid, pts[1].pid)

1659 8963
